In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.table import Table

In [3]:
# Function to fetch historical data
def get_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date)

# Set dates
end_date = datetime.today()
start_date = end_date - timedelta(days=366 * 2)  # 2 years of data for moving averages

def calculate_sharpe_ratio(stock_ticker, period, risk_free_rate=0.065):
    #start_dt = datetime.today() - timedelta(days = period * 40)
    #stock_data = pd.DataFrame(get_data(stock_ticker, start_dt, datetime.today()))[(period*-21)-1:]
    stock_data = data[stock_ticker.upper()][(period*-21)-1:]
    stock_data.sort_index(inplace=True)
    stock_data = stock_data.drop(stock_data.columns.difference(['Close']), axis=1)
    stock_data['daily_return'] = stock_data['Close'].pct_change()
    stock_data.dropna(subset=['daily_return'], inplace=True)
    sum_daily_return = stock_data['daily_return'].sum()
    std_daily_return = stock_data['daily_return'].std()
    period_std = std_daily_return * np.sqrt(period*21)
    sharpe_ratio = (sum_daily_return - risk_free_rate*(period/12)) / period_std
    return sharpe_ratio

def calculate_sortino_ratio(stock_ticker, period, risk_free_rate =0.065):
    #start_dt = datetime.today() - timedelta(days = period * 40)
    #stock_data = pd.DataFrame(get_data(stock_ticker, start_dt, datetime.today()))[(period*-21)-1:]
    stock_data = data[stock_ticker.upper()][(period*-21)-1:]
    stock_data.sort_index(inplace=True)
    stock_data = stock_data.drop(stock_data.columns.difference(['Close']), axis=1)
    stock_data['daily_return'] = stock_data['Close'].pct_change()
    stock_data.dropna(subset=['daily_return'], inplace=True)
    sum_daily_return = stock_data['daily_return'].sum()
    stock_data['daily_return_neg'] = stock_data['daily_return'].where(stock_data['daily_return']<0)
    std_daily_return = (stock_data['daily_return_neg']).std()
    period_std = std_daily_return * np.sqrt(period*21)
    sortino_ratio = (sum_daily_return - risk_free_rate*(period/12)) / period_std
    return sortino_ratio


def calc_return(stock_ticker, period):
    #start_dt = datetime.today() - timedelta(days = period * 40)
    #stock_data = pd.DataFrame(get_data(stock_ticker, start_dt, datetime.today()))[(period*-21)-1:]
    stock_data = data[stock_ticker.upper()][(period*-21)-1:]
    stock_data.sort_index(inplace=True)
    stock_data = stock_data.drop(stock_data.columns.difference(['Close']), axis=1)
    p_return = (stock_data['Close'][-1] / stock_data['Close'][0])-1
    return p_return

In [4]:
#db = pd.read_csv("/content/drive/MyDrive/fin_code/input/ind_nifty50list.csv")
#tickers = [i + ".NS" for i in db.Symbol]
tickers = ["ADANIENT.NS",	"ADANIPORTS.NS",	"APOLLOHOSP.NS",	"ASIANPAINT.NS",	"AXISBANK.NS",	"BAJAJ-AUTO.NS",	"BAJFINANCE.NS",	"BAJAJFINSV.NS",	"BPCL.NS",	"BHARTIARTL.NS",	"BRITANNIA.NS",	"CIPLA.NS",	"COALINDIA.NS",	"DIVISLAB.NS",	"DRREDDY.NS",	"EICHERMOT.NS",	"GRASIM.NS",	"HCLTECH.NS",	"HDFCBANK.NS",	"HDFCLIFE.NS",	"HEROMOTOCO.NS",	"HINDALCO.NS",	"HINDUNILVR.NS",	"ICICIBANK.NS",	"ITC.NS",	"INDUSINDBK.NS",	"INFY.NS",	"JSWSTEEL.NS",	"KOTAKBANK.NS",	"LTIM.NS",	"LT.NS",	"M&M.NS",	"MARUTI.NS",	"NTPC.NS",	"NESTLEIND.NS",	"ONGC.NS",	"POWERGRID.NS",	"RELIANCE.NS",	"SBILIFE.NS",	"SHRIRAMFIN.NS",	"SBIN.NS",	"SUNPHARMA.NS",	"TCS.NS",	"TATACONSUM.NS","TATAMOTORS.NS","TATASTEEL.NS",	"TECHM.NS",	"TITAN.NS",	"ULTRACEMCO.NS",	"WIPRO.NS"]

# Data dictionary to hold stock data
data = {}

# Fetch data for all tickers
for ticker in tickers:
    try:
        stock_data = get_data(ticker, start_date, end_date)
        if len(stock_data) > 0:
            data[ticker] = stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [5]:
# Create a DataFrame for summary
summary = []

# Analyze each stock
for ticker, df in data.items():
    try:
        # Calculate EMAs
        df['EMA100'] = df['Close'].ewm(span=100).mean()
        df['EMA200'] = df['Close'].ewm(span=200).mean()

        # Last 1-year return
        one_year_return = (df['Close'][-1] / df['Close'][-252] - 1) * 100

        # 52-week high
        high_52_week = df['Close'][-252:].max()
        within_20_pct_high = df['Close'][-1] >= high_52_week * 0.8

        # More than 50% up days in the last 6 months (126 trading days)
        six_month_data = df['Close'][-126:]
        up_days = (six_month_data.pct_change() > 0).sum()
        up_days_pct = up_days / len(six_month_data) * 100

        # Filtering criteria
        if (df['Close'][-1] >= df['EMA100'][-1] >= df['EMA200'][-1] and
            one_year_return >= 6.5 and
            within_20_pct_high and
            up_days_pct > 50):

            # Calculate returns
            return_12m = (df['Close'][-1] / df['Close'][-252] - 1) * 100
            return_9m = (df['Close'][-1] / df['Close'][-189] - 1) * 100
            return_6m = (df['Close'][-1] / df['Close'][-126] - 1) * 100
            return_3m = (df['Close'][-1] / df['Close'][-63] - 1) * 100

            summary.append({
                'Ticker': ticker,
                'Return_12M': calc_return(ticker,12)*100,
                'Return_9M': calc_return(ticker,9)*100,
                'Return_6M': calc_return(ticker,6)*100,
                'Return_3M': calc_return(ticker,3)*100,
                'Sharpe_12M': calculate_sharpe_ratio(ticker, 12),
                'Sharpe_9M': calculate_sharpe_ratio(ticker, 9),
                'Sharpe_6M': calculate_sharpe_ratio(ticker, 6),
                'Sharpe_3M': calculate_sharpe_ratio(ticker, 3),
                'Sortino_12M': calculate_sortino_ratio(ticker, 12),
                'Sortino_9M': calculate_sortino_ratio(ticker, 9),
                'Sortino_6M': calculate_sortino_ratio(ticker, 6),
                'Sortino_3M': calculate_sortino_ratio(ticker, 3)
            })
    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")

In [15]:
# Convert summary to DataFrame
df_summary = pd.DataFrame(summary,index=np.arange(1, len(summary)+1))

rat = ['Sortino', 'Sharpe'] # also add ['Return', 'Sortino', 'Sharpe']
per = [3,6,9,12]

for i in rat:
    for j in per:
        xx = i+"_"+str(j)+"M"
        df_summary[xx] = df_summary[xx].round(2)

for i in rat:
    arr = np.array([])
    fx = "Final_Rank_" + i
    for j in per:
        xx = i+"_"+str(j)+"M"
        rxx = "Rank_"+i+"_"+str(j)+"M"
        arr = np.append(arr,rxx)
        #arr = np.append(arr,arr)
        df_summary[rxx] = df_summary[xx].rank(method='max',ascending=False)

    df_summary[fx] = df_summary.loc[:,arr].sum(axis=1)

In [16]:
#Export as .csv
for i in rat:
    arr = np.array(["Ticker"])
    fx = "Final_Rank_" + i
    arr = np.append(arr,fx)
    for j in per:
        xx = i+"_"+str(j)+"M"
        rxx = "Rank_"+i+"_"+str(j)+"M"
        arr = np.append(arr,rxx)
        arr = np.append(arr,xx)

    df_sorted = df_summary[arr].sort_values(fx)
    df_sorted['Position'] = np.array([i for i in range(1, len(df_sorted)+1)])
    df_sorted = df_sorted.reset_index(drop=True).set_index(['Position'])
    
    # save the dataframe as a csv file
    df_sorted.to_csv("./outputs/nifty_list_{}_{}_sort.csv".format(datetime.date(datetime.today()),i))